In [8]:
using Pkg
Pkg.activate("..")

# Load module
push!(LOAD_PATH, "../src")
using WriterVerifier

using Random
Random.seed!(42);

  Activating project at `c:\Users\igorp\WriterVerifier`


In [2]:
# Load data
println("Loading images...")

image_folder = "../data/words"

# Load images
writers = load_images(image_folder; max_per_writer=10000);

Loading images...
Loaded 76 writers
f01: 857 images
d01: 960 images
g05: 225 images
e07: 1371 images
e02: 877 images
a04: 1918 images
p02: 1578 images
p06: 775 images
r02: 1359 images
h07: 3017 images
b04: 2196 images
k02: 971 images
d03: 145 images
d06: 2116 images
c03: 4435 images
b01: 1832 images
c04: 2320 images
f03: 291 images
c02: 1141 images
g06: 10000 images
l07: 1151 images
j07: 344 images
k04: 2429 images
e06: 1140 images
l03: 205 images
a03: 1652 images
r03: 1123 images
n06: 1593 images
j04: 1332 images
r06: 2111 images
n02: 1790 images
g03: 725 images
k01: 86 images
j06: 740 images
h04: 1252 images
m02: 1379 images
b05: 1182 images
g02: 299 images
k03: 520 images
c01: 249 images
f02: 864 images
p03: 2173 images
p01: 337 images
a05: 1943 images
d05: 456 images
m01: 1663 images
g04: 2059 images
h06: 613 images
f04: 2060 images
n01: 687 images
j01: 514 images
b02: 430 images
b03: 519 images
l04: 2331 images
b06: 1996 images
n04: 2377 images
e01: 1284 images
l01: 1610 images
a0

In [3]:
# Create pairs
pairs, labels = create_pairs(writers; positive=50, negative=50);

Creating positive pairs...
Creating negative pairs...
Created 100 pairs
Positive: 50
Negative: 50


In [ ]:
# Create model
model = create_model()
println("Model created")

# Test model
test_model(model);


Model created
Testing model...
Output shape: (2,)
Values: Float32[0.508, 0.499]
Range: [0.499, 0.508]
Loss: 0.6841


In [5]:
# Model training

trained_model, history = train_model!(
    model, pairs, labels;
    epochs=5, 
    batch_size=200,
    learning_rate=0.001
)

println("Training completed")

# Show final results
final_acc = round(history["val_acc"][end] * 100, digits=1)
println("Final accuracy: $(final_acc)%")

Starting training...
Training data: 80
Validation data: 20

Epoch 1/5
Train loss: 0.6973
Val loss: 0.6959
Val accuracy: 35.0%

Epoch 2/5
Train loss: 0.6989
Val loss: 0.695
Val accuracy: 45.0%

Epoch 3/5
Train loss: 0.7096
Val loss: 0.6957
Val accuracy: 30.0%

Epoch 4/5
Train loss: 0.6993
Val loss: 0.6967
Val accuracy: 40.0%

Epoch 5/5
Train loss: 0.6928
Val loss: 0.696
Val accuracy: 50.0%

Training completed!
Training completed
Final accuracy: 50.0%


In [21]:
# Testing on real examples

# Select random test examples
idx = randperm(length(pairs))[1:min(6, length(pairs))]
test_pairs = pairs[idx]
test_labels = labels[idx]

for i in 1:length(test_pairs)
    path1, path2 = test_pairs[i]
    true_label = test_labels[i]
    
    # Extract writer names
    writer1 = split(basename(path1), "-")[1]
    writer2 = split(basename(path2), "-")[1]
    
    # Test similarity
    similarity = test_similarity(trained_model, path1, path2)
    
    # Interpret results
    prediction = similarity > 0.5 ? "SAME" : "DIFFERENT"
    truth = true_label == 1 ? "SAME" : "DIFFERENT"
    correct = (similarity > 0.5) == (true_label == 1) ? "✓" : "X"
    
    println("$i. $writer1 vs $writer2")
    println("   Similarity: $(round(similarity, digits=3))")
    println("   Prediction: $prediction | Truth: $truth $correct")
    println()
end


1. m04 vs f07
   Similarity: 0.505
   Prediction: SAME | Truth: DIFFERENT X

2. d04 vs d04
   Similarity: 0.503
   Prediction: SAME | Truth: SAME ✓

3. b03 vs b03
   Similarity: 0.504
   Prediction: SAME | Truth: SAME ✓

4. n03 vs d01
   Similarity: 0.506
   Prediction: SAME | Truth: DIFFERENT X

5. k07 vs k07
   Similarity: 0.503
   Prediction: SAME | Truth: SAME ✓

6. f04 vs e01
   Similarity: 0.508
   Prediction: SAME | Truth: DIFFERENT X



In [7]:
# Model saving
try
    if !isdir("../models")
        mkdir("../models")
    end
    
    save_model(trained_model, "../models/model.jld2")
    
catch e
    println("Save error: $e")
end

Model saved: ../models/model.jld2
